# *ViVAE* and *ViScore* usage example

In this Jupyter notebook, we download a single-cell dataset from Zenodo, run basic pre-processing on in and make a simple 2-dimensional layout of the data using *ViVAE*.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
import ViVAE, ViScore, copy

### **1.** Import data

We start by importing a pre-processed [scRNA-seq dataset](https://singlecell.broadinstitute.org/single_cell/study/SCP162) from `./data` (make sure this has been downloaded via [Git LFS](https://docs.github.com/en/repositories/working-with-files/managing-large-files/installing-git-large-file-storage)).

For scRNA-seq datasets, we typically recommend using the first 50 components of the count data.
For cytometry data, use the pre-processed protein expression matrix (*ie.* post- compensation, transformation and batch effect correction, or whichever applies to your use-case).

In order to smooth (de-noise) the data, we use a k-NN graph: a pre-computed one is provided here, but a new one can be computed using the *PyNNDescent* nearest-neighbour search algorithm with `ViVAE.make_knn`.

**(If you want to use the *SQuadVAE* model without de-noising the input data, there is no need for the *k*-NNG.)**

In [ ]:
load = lambda dataset : [np.load(f'./data/{dataset}_{x}.npy', allow_pickle=True) for x in ['pc', 'knn', 'annot']]
pc, knn, annot = load('Shekhar')

### **2.** De-noise inputs

Nearest-neighbour-based de-noising of inputs (pre-smoothing) is done using the approximate *k*-NN graph computed earlier (see *Methods* section of the paper).

A single iteration with $\lambda$=1 and $k$=50 is applied here.
This is the default pre-smoothing set-up proposed for flow cytometry, CyTOF and scRNA-seq data.
If small populations are present, decrease $k$.
If working with less noisy (non-biological even?) data, experiment with smaller values for $\lambda$.

(For post-smoothing, if used, multiple iterations with $\lambda$ around 0.01 is recommended.
For a quantitative evaluation of how these settings work with your dataset, use *ViScore*!)

In [ ]:
pc_s = ViVAE.smooth(x=pc, knn=knn, k=50, coef=1., n_iter=1)

### **3.** Train an *SQuadVAE* model

We train *SQuadVAE* (VAE with a quartet loss regularisation term) on the PCs.

In [ ]:
model = ViVAE.ViVAE(full_dim=pc_s.shape[1], latent_dim=2)

In [ ]:
model.fit(pc_s)

### **4.** Create embedding and plot it

The trained model is then used to create the lower-dimensional embedding of the dataset we trained on (or, alternatively, a similar enough or extended dataset).

*ViVAE* also has a plotting function that quickly visualises the embedding (or its first two components) with cell population annotation when available.

In [ ]:
ld = model.transform(pc_s)

In [ ]:
palette = [
    'grey',    '#1CE6FF', '#FF34FF', '#FF4A46', '#008941', '#006FA6', '#A30059', '#7A4900', '#dedb8c', '#63FFAC', '#B79762', '#004D43', '#8FB0FF', '#997D87',
    '#5A0007', '#809693', '#1B4400', '#4FC601', '#3B5DFF', '#4A3B53', '#FF2F80', '#61615A', '#BA0900', '#6B7900', '#00C2A0', '#FFAA92', '#FF90C9', '#B903AA',
    '#D16100', '#DDEFFF', '#000035', '#7B4F4B', '#A1C299', '#300018', '#0AA6D8', '#013349', '#00846F', '#372101', '#FFB500', '#C2FFED', '#A079BF', '#CC0744',
    '#C0B9B2', '#C2FF99', '#001E09', '#00489C', '#6F0062', '#0CBD66', '#EEC3FF', '#456D75', '#B77B68', '#7A87A1', '#788D66', '#885578', '#FAD09F', '#FF8A9A',
    '#D157A0', '#BEC459', '#456648', '#0086ED', '#886F4C', '#34362D', '#B4A8BD', '#00A6AA', '#452C2C', '#636375', '#A3C8C9', '#FF913F', '#938A81', '#575329',
    '#00FECF', '#B05B6F', '#8CD0FF', '#3B9700', '#04F757', '#C8A1A1', '#1E6E00', '#7900D7', '#A77500', '#6367A9', '#A05837', '#6B002C', '#772600', '#D790FF',
    '#9B9700', '#549E79', '#FFF69F', '#201625', '#72418F', '#BC23FF', '#99ADC0', '#3A2465', '#922329', '#5B4534', '#FDE8DC', '#404E55', '#0089A3', '#CB7E98',
    '#A4E804', '#324E72', '#6A3A4C'
]

In [ ]:
ViVAE.plot(proj=ld, annot=annot, unassigned='nan', figsize=(6,5), dpi=80, point_size=.01, title='Shekhar retina dataset embedding', palette=palette)

### **5.** Use structure-preservation metrics as unsupervised score

Using *ViScore*, we can calculate the local and global structure-preservation index ($S_{L}$ and $S_{G}$, respectively).

For reference, we can compare the *ViVAE* embedding with the first two PCs of the original data.
(To compare to alternative non-linear dimensionality reduction methods, use their resulting embeddings instead.)

In [ ]:
score_vivae = ViScore.score(hd=pc, ld=ld)
score_pca = ViScore.score(hd=pc, ld=pc[:,range(2)])

In [ ]:
print(f'ViVAE embedding scores\n\tLocal:\t{score_vivae["Sl"]:.3f}\n\tGlobal:\t{score_vivae["Sg"]:.3f}\nFirst 2 PCs scores\n\tLocal:\t{score_pca["Sl"]:.3f}\n\tGlobal:\t{score_pca["Sg"]:.3f}')

### **6.** Use supervised evaluation to describe population-wise embedding errors

*ViScore* can also help qualify and quantify the nature of embedding distorion as it pertains to any given population, to limit misinterpretation of dimensionality reduction.

In [ ]:
nc_hd = ViScore.neighbourhood_composition(X=pc, pop='BC6', annot=annot, exclude='nan')
nc_ld = ViScore.neighbourhood_composition(X=ld, pop='BC6', annot=annot, exclude='nan')

In [ ]:
palette_without_bc6 = copy.deepcopy(palette)
del palette_without_bc6[11]

In [ ]:
plot_hd = ViScore.neighbourhood_composition_plot(nc=nc_hd, palette=palette_without_bc6)
plt.show(plot_hd)

In [ ]:
plot_ld = ViScore.neighbourhood_composition_plot(nc=nc_ld, palette=palette_without_bc6)
plt.show(plot_ld)